In [1]:
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import SelectKBest, chi2

In [2]:
# SELECCIÓN DEL MODELO Y ENTRENAMIENTO

# Lectura del dataset

dataset = pd.read_csv('C:/Users/toled/OneDrive/Documentos/MATEMÁTICAS/TFG/Dataset NBA.csv',sep=";", encoding="ISO 8859-1")
dataset = dataset.drop('YEAR', 1)
dataset = dataset.drop('Team', 1)
dataset = dataset.drop('Arena', 1)
dataset = dataset.drop('Attend.', 1)
dataset = dataset.drop('PW', 1)
dataset = dataset.drop('L%', 1)
dataset = dataset.drop('PL', 1)
dataset = dataset.drop('Champion', 1)
dataset = dataset.drop('Rk', 1)

dataset.head()

C:\Users\toled\AppData\Local\Temp/ipykernel_11012/1073130396.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataset = dataset.drop('YEAR', 1)
C:\Users\toled\AppData\Local\Temp/ipykernel_11012/1073130396.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataset = dataset.drop('Team', 1)
C:\Users\toled\AppData\Local\Temp/ipykernel_11012/1073130396.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataset = dataset.drop('Arena', 1)
C:\Users\toled\AppData\Local\Temp/ipykernel_11012/1073130396.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataset = dataset.drop('Attend.', 1)
C:\Users\toled\AppData\Local\Temp/ipykernel_11012/1073130396

,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,...,3PAr,TS%,OFeFG%,OfTOV%,ORB%,Of FT/FGA,DEFeFG%,DefTOV%,DRB%,Def FT/FGA
0,82,240.3,41.5,88.3,47.0,12.9,34.4,37.4,28.6,53.9,...,390,58.1,54.3,10.8,23.0,205,54.3,11.5,76.9,177
1,82,240.6,41.7,85.6,48.7,1.5,5.0,30.2,40.2,80.6,...,59,54.7,49.6,13.5,36.3,277,50.9,14.0,65.5,254
2,82,242.7,42.6,91.2,46.7,2.6,8.2,31.3,40.0,83.0,...,90,51.9,48.1,13.0,33.6,203,49.7,13.2,68.3,209
3,82,242.7,34.9,78.5,44.4,4.9,13.9,35.2,30.0,64.5,...,177,52.7,47.6,15.7,27.9,248,46.9,12.1,70.4,222
4,82,242.1,42.0,86.8,48.4,0.9,2.9,31.1,41.1,84.0,...,33,53.6,48.9,15.3,32.5,250,48.5,15.2,63.8,249


In [4]:
# create a list of our conditions
conditions = [
    (dataset['W%'] <= 17/82),
    (dataset['W%'] > 17/82) & (dataset['W%'] <= 25/82),
    (dataset['W%'] > 25/82) & (dataset['W%'] <= 32/82),
    (dataset['W%'] > 32/82) & (dataset['W%'] <= 38/82),
    (dataset['W%'] > 38/82) & (dataset['W%'] <= 44/82),
    (dataset['W%'] > 44/82) & (dataset['W%'] <= 51/82),
    (dataset['W%'] > 51/82) & (dataset['W%'] <= 58/82),
    (dataset['W%'] > 58/82) & (dataset['W%'] <= 65/82),
    (dataset['W%'] > 65/82)
    ]

# create a list of the values we want to assign for each condition
values = ['abysmal', 'tanking', 'poor', 'mediocre', 'decent','good','contender','top','goat']

# create a new column and use np.select to assign values to it using our lists as arguments
dataset['tier'] = np.select(conditions, values)

In [18]:
# Preprocesado

# Se separa la columna a predecir
X = dataset.iloc[:,dataset.columns != 'W%']
X = X.iloc[:,X.columns != 'tier']
y = dataset.iloc[:,-1]
#y = np.array([0]*1194)

# En este ejemplo, se reescalan con un MinMaxScaler
sc = MinMaxScaler()
sc.fit(X) # Se ajusta el reescalador
X = sc.transform(X)

In [19]:
kb = SelectKBest(chi2, k=8).fit(X,y)

In [20]:
kb.get_support()

array([False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True,  True,
        True, False, False, False,  True,  True, False, False, False,
       False, False, False, False])

In [21]:
dataset = dataset.drop('W%',1)
dataset = dataset.drop('tier',1)

dataset.columns

C:\Users\toled\AppData\Local\Temp/ipykernel_11012/3765973981.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataset = dataset.drop('W%',1)
C:\Users\toled\AppData\Local\Temp/ipykernel_11012/3765973981.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataset = dataset.drop('tier',1)


Index(['G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%',
       'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', 'Age', 'MOV', 'SOS', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr',
       'TS%', 'OFeFG%', 'OfTOV%', 'ORB%', 'Of FT/FGA', 'DEFeFG%', 'DefTOV%',
       'DRB%', 'Def FT/FGA'],
      dtype='object')

In [22]:
kb.scores_

array([ 0.29721652,  3.20228996,  4.38517391,  0.51427548, 13.55024393,
        2.64475165,  1.5904462 ,  1.46159657,  0.96872241,  1.21645451,
       15.57663653,  2.1685361 ,  1.32607148,  2.37317861,  1.11647927,
        7.25032558,  3.58307329,  6.8932232 ,  1.62138611,  3.52914726,
        4.43204363,  3.0217801 ,  6.52484099, 19.29130136, 56.35035321,
       15.12699639, 20.49757152, 17.70315758,  0.38061148,  2.0603868 ,
        1.81282094, 18.35066054, 17.148788  ,  3.68117324,  0.75214744,
        2.98621772, 12.70781985,  1.0230226 ,  3.43573735,  4.84749102])

In [28]:
A = sum(kb.scores_)

In [27]:
B = 56.35035321+20.49757152+19.29130136+18.35066054+17.70315758+17.148788+15.57663653+15.12699639

In [29]:
B/A

0.6264539496591334